In [47]:
import torch
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torch.nn as nn
import torchvision.models as models

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

LR=0.001
EPOCHS=10

In [50]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

train = ImageFolder(root='./train', transform=transform)

train_dl = DataLoader(train, batch_size=512, shuffle=True)

In [56]:
train

Dataset ImageFolder
    Number of datapoints: 250874
    Root location: ./train
    StandardTransform
Transform: Compose(
               RandomHorizontalFlip(p=0.5)
               ToTensor()
           )

In [4]:
class ResNetModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.resnet = models.resnet50(pretrained=True)
    for param in self.resnet.parameters():
      param.requires_grad = False

    self.resnet.fc = nn.Sequential(
        nn.Linear(2048, 163),
    )
  
  def forward(self, x):
    return self.resnet(x)

In [5]:
model = ResNetModel()
model.load_state_dict(torch.load('quick_lol.pth', map_location=torch.device('cpu')))

c:\Users\tolst\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\tolst\AppData\Local\Programs\Python\Python39\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


<All keys matched successfully>

In [ ]:
from PIL import Image

image = Image.open('image.png').convert('RGB').resize((300, 400))
image

In [43]:
model.eval()
image = Image.open('image.png').convert('RGB').resize((400, 300))
image = transforms.ToTensor()(image).unsqueeze(0)
predict = model(image).argsort(dim=1, descending=True)

In [44]:
for i in predict[0][:10]:
    print(train_dl.dataset.find_classes('./train')[0][i])

Серафина
Кай'Са
Тарик
Тимо
Акшан
Сайлас
Фиора
Гвен
Кассиопея
Седжуани
